In [ ]:
! pip install pandas openai beautifulsoup4 nltk matplotlib wordcloud tiktoken markdown spacy transformers 

In [8]:
##Required Packages 
import os
import sys
import pandas as pd
import re
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import DutchStemmer
from nltk.corpus import alpino
import matplotlib.pyplot as plt
import spacy
import tiktoken
import json
import numpy as np
from scipy.special import softmax
import markdown


from lingua import Language, LanguageDetectorBuilder
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer
####
nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('alpino')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\samge\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\samge\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package alpino to
[nltk_data]     C:\Users\samge\AppData\Roaming\nltk_data...
[nltk_data]   Package alpino is already up-to-date!


True

In [25]:
#CHANGE FILENAME
data = pd.read_csv("Reviews.csv")


C:\Users\samge\AppData\Local\Temp\ipykernel_9112\1363499268.py:2: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("Reviews.csv")


In [26]:
data = data.dropna(subset = ["Review general"])
def word_count(text):
    return len(str(text).split())

# Create a new column for word counts (optional, for inspection)
data['review_general_word_count'] = data['Review general'].apply(word_count)

# Filter rows where word count is 5 or more
data = data[data['review_general_word_count'] >= 3]
data = data.sample(n=1000)


,Retailer ID,Type,Review ID,Shop ID,Review date,NPS,Review general,NPS driver,NPS driver explanation,review_general_word_count,Language
517681,6,Food,a414131f-4cca-4b7a-952f-4de6f27ef00c,21769,08/07/2020 08:34,5,Broodjes waren heel dun belegd er lag meer ble...,NaN,NaN,12,Language.DUTCH
8595,1,Discounter,d784fbac-f6b6-4a66-a00b-4816ab3b6f8c,87833,28/08/2024 17:38,10,"Very good. Good price, friendly and dedicated ...",Afrekenen,NaN,22,Language.ENGLISH
129971,1,Discounter,79ff0ac4-f48f-41f9-902c-89496f2d8dc9,87899,23/02/2024 11:57,10,"Very good store, very clean and neatly arrange...",Uitstraling,NaN,25,Language.ENGLISH
207926,2,DIY,3b49ae1b-b7db-11e7-a3aa-c21916f619ea,1699,02/11/2016 22:40,10,De meeste producten zijn aanwezig en meestal i...,NaN,NaN,24,Language.DUTCH
517253,6,Food,a31f9983-06f0-466a-8fe2-44072bb9053c,2993,14/06/2018 11:20,7,De soep was een beetje aan de lauwe kant. Maar...,NaN,NaN,26,Language.DUTCH
...,...,...,...,...,...,...,...,...,...,...,...
141864,1,Discounter,6bdafb9d-8d36-4ec6-a1f2-00fcc9a1ee44,35789,06/04/2024 12:00,10,"My purchase went very well, I found all the pr...",Weekactie,NaN,12,Language.ENGLISH
411206,2,DIY,ec27c5fb-74ff-4238-a51b-28c9fbb91ad4,1754,07/04/2023 14:59,9,Ik vind het jammer dat al mijn oude punten\r\n...,NaN,NaN,29,Language.DUTCH
432185,6,Food,074fffd5-1c2c-4b90-9119-237f408da8c3,127686,09/04/2017 19:24,7,In Lelystad there are not much to choose from,NaN,NaN,9,Language.ENGLISH
64970,1,Discounter,c6884e52-6d36-4c85-aaa1-61a27b2e08df,36998,17/04/2024 10:41,9,I left the store happy again,Klantvriendelijkheid,NaN,6,Language.ENGLISH


In [28]:
languages = [Language.ENGLISH, Language.DUTCH]
detector = LanguageDetectorBuilder.from_languages(*languages).build()
def detect_language(text):
    lang = detector.detect_language_of(text)
    return lang
        

data["Language"] = data["Review general"].apply(detect_language)



In [30]:

dutch_reviews = data[data["Language"] == Language.DUTCH]

nlp = spacy.load("nl_core_news_sm")
def remove_proper_nouns_spacy(text):
    doc = nlp(text)
    return ' '.join([token.text for token in doc if token.pos_ != 'PROPN'])

# Apply the function to all rows in "Review general"
dutch_reviews["Review general"] = dutch_reviews["Review general"].apply(remove_proper_nouns_spacy)
dutch_reviews["Language"] = dutch_reviews["Review general"].apply(detect_language)
english_reviews = data[data["Language"] == Language.ENGLISH]
english_reviews['P_review'] = english_reviews['Review general'].apply(preprocess_English)
dutch_reviews['P_review'] = dutch_reviews['Review general'].apply(preprocess_Dutch)

english_reviews.to_csv("English_Reviews.csv")
dutch_reviews.to_csv("Dutch_Reviews.csv")

C:\Users\samge\AppData\Local\Temp\ipykernel_9112\1184429679.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dutch_reviews["Review general"] = dutch_reviews["Review general"].apply(remove_proper_nouns_spacy)
C:\Users\samge\AppData\Local\Temp\ipykernel_9112\1184429679.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dutch_reviews["Language"] = dutch_reviews["Review general"].apply(detect_language)
C:\Users\samge\AppData\Local\Temp\ipykernel_9112\1184429679.py:12: SettingWithCopyWarning: 
A value is tr

In [22]:
######################################
#### DUTCH LANGUAGE PREPROCESSING ####
######################################

# Load the Dutch spaCy model
nlp = spacy.load("nl_core_news_sm")

# Initialize Dutch stopwords
stop_words = set(stopwords.words('dutch'))

def preprocess_Dutch(text):
    """
    Cleans, normalizes, and applies lemmatization to Dutch text.
    
    :param text: A string of text to preprocess.
    :return: Preprocessed text as a string.
    """
    # Remove HTML tags
    text = BeautifulSoup(text, "html.parser").get_text()
    
    # Remove URLs and any non-textual content
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation and special characters
    
    # Normalize text
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\d+', '', text)  # Remove numbers
    
    # Tokenize and remove stop words
    tokens = word_tokenize(text, language='dutch')
    tokens = [word for word in tokens if word not in stop_words]
    
    # Apply lemmatization using spaCy
    doc = nlp(' '.join(tokens))
    lemmas = [token.lemma_ for token in doc]
    
    # Rejoin tokens into a single string
    return ' '.join(lemmas)




In [23]:
######################################
#### English LANGUAGE PREPROCESSING ##
######################################

# Load the English spaCy model
nlp = spacy.load("en_core_web_sm")

# Initialize English stopwords
stop_words = set(stopwords.words('english'))

def preprocess_English(text):
    """
    Cleans, normalizes, and applies lemmatization to English text.
    
    :param text: A string of text to preprocess.
    :return: Preprocessed text as a string.
    """
    # Remove HTML tags
    text = BeautifulSoup(text, "html.parser").get_text()
    
    # Remove URLs and any non-textual content
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation and special characters
    
    # Normalize text
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'\d+', '', text)  # Remove numbers
    
    # Tokenize and remove stop words
    tokens = word_tokenize(text, language='english')
    tokens = [word for word in tokens if word not in stop_words]
    
    # Apply lemmatization using spaCy
    doc = nlp(' '.join(tokens))
    lemmas = [token.lemma_ for token in doc]
    
    # Rejoin tokens into a single string
    return ' '.join(lemmas)

In [31]:
likelihood_df = pd.read_csv("dict.csv", na_values=[], keep_default_na=False)
likelihood_df["words"] = likelihood_df["words"].astype("string")
likelihood_df.dropna()
likelihood_df.reset_index()
prior_probabilities = pd.read_csv("probs.csv", index_col=["NPS driver"])
prior_probabilities = prior_probabilities.sort_index()
prior_probabilities = np.array(prior_probabilities).flatten()
lexicon = pd.read_csv("Lexicon.csv")
lexicon["0"] = lexicon["0"].astype("string")
lexicon = lexicon["0"].to_list()
lexicon_fixed = []
for word in lexicon:
    word = str(word).lower()
    print(word)
    lexicon_fixed.append(word)


aaa
aachenrichterich
aalen
aalsmeer
aalst
aarschot
abandon
abandonment
abbeville
ability
able
abn
abnormal
abnormally
abrand
abrasive
abroad
abrupt
absence
absent
absolute
absolutely
absorb
absorber
absorption
absurd
abundance
abuse
abusive
abysmal
accent
accept
acceptability
acceptable
acces
access
accessibility
accessible
accessorie
accessory
accident
accidentally
accidentprone
accommodate
accompanied
accompany
accord
accordance
accordingly
accordinglyä
account
accountability
accountable
accumulate
accumulation
accumulator
accurate
accuse
accustomed
ache
achieve
acid
acknowledge
acne
acquaintance
acquire
across
acrtion
acrylic
act
actin
actio
action
actionapp
actionpacke
actionäù
activate
active
activity
actual
actually
ad
adam
adapt
adapter
add
addicted
addiction
addition
additional
additionally
additive
address
addressed
adequate
adequately
adhd
adhere
adhesive
adida
adjacent
adjust
adjustable
adjustment
administrative
admit
admittedly
ado
adria
adrift
ads
adult
advance
advantage
a

In [53]:
prior_probabilities = np.full(8, 1/8)
prior_probabilities

array([0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125])

In [54]:
def compute_driver_prob(prior, corpus_in, dict, p_w_given_c, BIGRAM, TOT_DIMENSIONS):
    if not isinstance(corpus_in, str):
        corpus_in = str(corpus_in)
    if p_w_given_c['words'].isna().any():
        p_w_given_c = p_w_given_c.dropna(subset=['words'])    

    # Validate the vocabulary (dict)
    if not all(isinstance(word, str) for word in dict):
        raise ValueError("Vocabulary (dict) contains non-string entries.")
    vec = CountVectorizer(vocabulary=dict, lowercase=True, ngram_range=(1, BIGRAM))
    word_matrix = vec.fit_transform([corpus_in]).toarray()

    # Check if there are any relevant words in the review, if there are, treat them. 
    # If not, use the prior.
    if word_matrix.sum() == 0:
        posterior = prior
    else:
        # Positions in word matrix that have words from this review
        word_matrix_indices = np.where(word_matrix > 0)[1]
        posterior = np.zeros(TOT_DIMENSIONS)
        

        # Loop around words found in review
        for word_matrix_index in word_matrix_indices:
            word = vec.get_feature_names_out()[word_matrix_index]
            p_w_given_c_index = np.where(p_w_given_c.words == word)[0]
            if len(p_w_given_c_index) == 0:
                print(f"Word '{word}' not found in likelihood DataFrame. Skipping...")
                vec_likelihood = prior
                continue
            else:
                p_w_given_c_index = p_w_given_c_index[0]
            p_w_given_c_index = np.where(p_w_given_c.words == word)[0][0]
            vec_likelihood = np.array([p_w_given_c.Afrekenen[p_w_given_c_index], 
                                       p_w_given_c.Anders[p_w_given_c_index],
                                       p_w_given_c.Assortiment[p_w_given_c_index],
                                       p_w_given_c.Klantvriendelijkheid[p_w_given_c_index],
                                       p_w_given_c.Prijs[p_w_given_c_index],
                                       p_w_given_c.Service[p_w_given_c_index],
                                       p_w_given_c.Uitstraling[p_w_given_c_index],
                                       p_w_given_c.Weekactie[p_w_given_c_index]])
            

             # Looping around occurrences | word
            for i in range(word_matrix[0, word_matrix_index]):
                numerat = prior * vec_likelihood
                denomin = prior.dot(vec_likelihood)
                posterior = numerat / denomin
                

                if np.sum(posterior) > 1.01:
                    raise Exception('ERROR')
    
                            
    return {'posterior_': posterior}

In [48]:
english_reviews = pd.read_csv("English_Reviews.csv")


In [62]:

category_names = [
"Afrekenen", "Anders", "Assortiment", "Klantvriendelijkheid",
"Prijs", "Service", "Uitstraling", "Weekactie"
]
english_reviews["P_review"] = english_reviews["P_review"].astype("string")
for review_index in range(len(english_reviews)):
    text = str(english_reviews["P_review"].iloc[review_index])
    TOT_DIMENSIONS = 8
    posterior_driver = compute_driver_prob(prior_probabilities, text, dict=lexicon_fixed, p_w_given_c=likelihood_df, BIGRAM=2, TOT_DIMENSIONS=TOT_DIMENSIONS)
    english_reviews.loc[review_index, "prob_Afrekenen"] = posterior_driver["posterior_"][0]
    english_reviews.loc[review_index, "prob_Anders"] = posterior_driver["posterior_"][1]
    english_reviews.loc[review_index, "prob_Assortiment"] = posterior_driver["posterior_"][2]
    english_reviews.loc[review_index, "prob_Klantvriendelijkheid"] = posterior_driver["posterior_"][3]
    english_reviews.loc[review_index, "prob_Prijs"] = posterior_driver["posterior_"][4]
    english_reviews.loc[review_index, "prob_Service"] = posterior_driver["posterior_"][5]
    english_reviews.loc[review_index, "prob_Uitstraling"] = posterior_driver["posterior_"][6]
    english_reviews.loc[review_index, "prob_Weekactie"] = posterior_driver["posterior_"][7]
    english_reviews.loc[review_index, "Pred_Driver"] = category_names[np.argmax(posterior_driver["posterior_"])]
    if posterior_driver["posterior_"][np.argmax(posterior_driver["posterior_"])] < 0.35:
        english_reviews.loc [review_index, "Pred_Driver"] = "Unknown"
 


Word 'true' not found in likelihood DataFrame. Skipping...
Word 'true' not found in likelihood DataFrame. Skipping...
